# Model Development

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report as cr

In [17]:
# Load the training data variables
%store -r X
%store -r Y

In [18]:
X.shape, len(Y)

((347, 4096), 347)

In [47]:
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=0.2, random_state=0)

In [51]:
C = 1
pipeline = Pipeline([("scaler", StandardScaler()), ("svc", SVC(kernel="rbf", C=C))])
pipeline.fit(X_train, Y_train)
pipeline.score(X_test, Y_test)

0.8714285714285714